In [1]:
% pwd

u'/home/giltrapo/Master_Data_Science/TFM/3_Data_Munging'

In [2]:
import pandas as pd
import numpy as np
import pickle
import re
import difflib
from unidecode import unidecode
import os

### Community of Madrid data munging

In [3]:
# Load list of school codes.

with open("../2_Collect_data/Files/school_tables_collection", "rb") as f:
    school_tables_collection = pickle.load(f)

In [4]:
# Check schools without tables.

[(key, len(value)) for key, value in school_tables_collection.items() if len(value) == 0]

[(u'CP INF-PRI "angel nieto"_1', 0),
 (u'CP INF-PRI "angel nieto"_0', 0),
 (u'CP INF-PRI "hortaleza"_1', 0),
 (u'CP INF-PRI "hortaleza"_0', 0),
 (u'CP INF-PRI "blas de lezo"_1', 0),
 (u'CP INF-PRI "alfredo di stefano"_0', 0),
 (u'CP INF-PRI "alfredo di stefano"_1', 0)]

In [5]:
# Extract keys, depending on the type of table.
regex = re.compile("_0$")
school_tables_collection_keys_0 = \
    [key for key in school_tables_collection if regex.search(key)]
school_tables_collection_keys_1 = \
    [key for key in school_tables_collection if not regex.search(key)]

In [6]:
# Split dictionary, depending on the type of table.

school_tables_collection_0 = \
    {keys_0: school_tables_collection[keys_0] for keys_0 in school_tables_collection_keys_0}
school_tables_collection_1 = \
    {keys_1: school_tables_collection[keys_1] for keys_1 in school_tables_collection_keys_1}

In [7]:
# Create dataframe with number of students by year and school.

df_students = pd.DataFrame()

for key, value in school_tables_collection_0.items():
    try:
        df_temp = value[0].iloc[2:3,1:]
        df_temp.index = [key.split('"')[1]]
        df_students = df_students.append(df_temp)
    except:
        print key.split('"')[1]
        df_temp = pd.DataFrame(index = [key.split('"')[1]])
        df_students = df_students.append(df_temp)

alfredo di stefano
hortaleza
blas de lezo
angel nieto


In [8]:
df_students.reset_index(inplace = True)
df_students.columns.values[0] = 'key'
df_students.shape

(247, 6)

In [9]:
# Create dataframe with applications submitted/accepted by year and school.

df_enrollments = pd.DataFrame()

for key, value in school_tables_collection_1.items():
    try:
        df_temp = value[0][:2]
        df_temp.columns.values[0] = 'Status'
        df_temp = df_temp.assign(Pivot = (1, 1))
        df_temp = df_temp.pivot(index = "Pivot", columns = "Status")
        df_temp.columns = df_temp.columns.map(" ".join)
        df_temp = df_temp.reset_index()
        del df_temp["Pivot"]
        df_temp.index = [key.split('"')[1]]
        df_enrollments = df_enrollments.append(df_temp)
    except:
        print key.split('"')[1]
        df_temp = pd.DataFrame(index=[key.split('"')[1]])
        df_enrollments = df_enrollments.append(df_temp)

hortaleza
blas de lezo
alfredo di stefano
angel nieto


In [10]:
df_enrollments.reset_index(inplace = True)
df_enrollments.columns.values[0] = 'key'
df_enrollments.shape

(247, 11)

In [11]:
# Merge number of students and applications dataframes.

df_community = pd.merge(df_students, df_enrollments, on = "key")
df_community.shape

(247, 16)

### City Hall data munging

In [12]:
# Load City Hall file with geo-information about public primary schools.

df_cityhall = pd.read_csv("csv_files/202311-0-colegios-publicos.csv",
                          delimiter = ";",
                          usecols = [1, 20, 21, 24, 25],
                          encoding = "ISO-8859-1")

In [13]:
df_cityhall["NOMBRE"] = df_cityhall["NOMBRE"].str.replace("Colegio P\xfablico ", "")
df_cityhall = df_cityhall.assign(key = \
                                 [unidecode(string) for string in df_cityhall["NOMBRE"].values])
df_cityhall["key"] = df_cityhall["key"].str.lower()
df_cityhall["key"] = df_cityhall["key"].str.replace("(", "")
df_cityhall["key"] = df_cityhall["key"].str.replace(")", "")
df_cityhall.columns = ["Colegio", "Barrio", "Distrito", "Latitud", "Longitud", "key"]
df_cityhall["Colegio"] = [item.encode("utf8") for item in df_cityhall["Colegio"]]
df_cityhall["Barrio"] = [item.encode("utf8") for item in df_cityhall["Barrio"]]
df_cityhall["Distrito"] = [item.encode("utf8") for item in df_cityhall["Distrito"]]

In [14]:
# The school names in "df_community" do not exactly match school names on the City Hall file.
# We will use "difflib" library to create a variable with the school name on "df_community"
# that most resembles the school name on the City Hall file.

for i, school in enumerate(df_cityhall["key"]):
    try:
        df_cityhall.loc[i, "key"] = difflib.get_close_matches(school, df_community["key"], 
                                                              n = 1)[0]
    except:
        df_cityhall.loc[i, "key"] = np.NaN[0]

### Merging Community and City Hall data

In [15]:
# Merge dataframes by school name

df_schools = pd.merge(df_cityhall, df_community, on = "key", how = "outer")
df_schools.loc[242:,"Colegio"] = ["Alfredo di Stefano",
                                  "República de Panamá nº 1",
                                  "Concepción Arenal",
                                  "Ángel Nieto",
                                  "Plácido Domingo"]
df_schools.loc[242:,"Barrio"] = ["VALDEFUENTES",
                                  "ARCOS",
                                  "COMILLAS",
                                  "SANTA EUGENIA",
                                  "ATOCHA"]
df_schools.loc[242:,"Distrito"] = ["HORTALEZA",
                                  "SAN BLAS-CANILLEJAS",
                                  "CARABANCHEL",
                                  "VILLA DE VALLECAS",
                                  "ARGANZUELA"]
del df_schools["key"]

### Municipal register data munging

In [16]:
# The enrollment process begins in May, so we will select children with ages between 5 and 11
# at that time. In addition, since there are only data on the City Hall's website until 2014,
# the data for 2013 and 2012 will be projected from the file of 2014, choosing an age range
# of 6 to 12 for 2013 and 7 to 13 for 2012.

files = ["csv_files/" + f for f in os.listdir("csv_files/") if f.startswith("Rango")]
files.sort()

In [17]:
df_population_primary = df_temp = pd.read_csv("csv_files/Rango_Edades_Seccion_201405.csv",
                                              delimiter = ";")
df_population_primary = df_population_primary.groupby(["DESC_DISTRITO", "DESC_BARRIO"])\
                                             .count().reset_index()\
                                             .iloc[:,[0,1]]

for file in files:
    if file[31:35] == "2014":
        year = 2012
        lim_inf = 6
        lim_sup = 14
        while (year < 2015):
            df_temp = pd.read_csv(file, delimiter = ";")
            df_temp = df_temp[(df_temp["COD_EDAD_INT"] > lim_inf) & 
                              (df_temp["COD_EDAD_INT"] < lim_sup)]
            df_temp[year] = df_temp.fillna(0)["EspanolesHombres"] +\
                            df_temp.fillna(0)["EspanolesMujeres"] +\
                            df_temp.fillna(0)["ExtranjerosHombres"] +\
                            df_temp.fillna(0)["ExtranjerosMujeres"]
            df_temp = df_temp.groupby(["DESC_DISTRITO", "DESC_BARRIO"])[year]\
                             .sum()\
                             .reset_index()\
                             .sort_values("DESC_DISTRITO")
            df_population_primary = pd.merge(df_population_primary, 
                                             df_temp, 
                                             on = ["DESC_DISTRITO", "DESC_BARRIO"], 
                                             how = "outer")
            year = year + 1
            lim_inf = lim_inf - 1
            lim_sup = lim_sup -1
            
    else:
        year = file[31:35]
        df_temp = pd.read_csv(file, delimiter = ";", encoding = "ISO-8859-1")
        df_temp = df_temp[(df_temp["COD_EDAD_INT"] > 4) & (df_temp["COD_EDAD_INT"] < 12)]
        df_temp[year] = df_temp.fillna(0)["EspanolesHombres"] +\
                        df_temp.fillna(0)["EspanolesMujeres"] +\
                        df_temp.fillna(0)["ExtranjerosHombres"] +\
                        df_temp.fillna(0)["ExtranjerosMujeres"]
        df_temp = df_temp.groupby(["DESC_DISTRITO", "DESC_BARRIO"])[year]\
                         .sum()\
                         .reset_index()\
                         .sort_values(["DESC_DISTRITO", "DESC_BARRIO"])
        df_temp["DESC_DISTRITO"] = [item.encode("utf8") for item in df_temp["DESC_DISTRITO"]]
        df_temp["DESC_BARRIO"] = [item.encode("utf8") for item in df_temp["DESC_BARRIO"]]
        df_population_primary = pd.merge(df_population_primary, 
                                         df_temp, 
                                         on = ["DESC_DISTRITO", "DESC_BARRIO"], 
                                         how = "outer")

In [18]:
df_population_primary["DESC_DISTRITO"] = \
    [item.strip() for item in df_population_primary["DESC_DISTRITO"]]
df_population_primary["DESC_BARRIO"] = \
    [item.strip() for item in df_population_primary["DESC_BARRIO"]]

### Merging all dataframes

In [19]:
df_total = pd.merge(df_schools,
                    df_population_primary,
                    left_on = ["Distrito", "Barrio"],
                    right_on = ["DESC_DISTRITO", "DESC_BARRIO"])

In [20]:
del df_total["DESC_BARRIO"]
del df_total["DESC_DISTRITO"]

In [21]:
df_total.columns = ['Colegio',
                    'Barrio',
                    'Distrito',
                    'Latitud',
                    'Longitud',
                    'Alumnos Primaria 2012-2013',
                    'Alumnos Primaria 2013-2014',
                    'Alumnos Primaria 2014-2015',
                    'Alumnos Primaria 2015-2016',
                    'Alumnos Primaria 2016-2017',
                    'Admisiones 2012-2013',
                    'Solicitudes 2012-2013',
                    'Admisiones 2013-2014',
                    'Solicitudes 2013-2014',
                    'Admisiones 2014-2015',
                    'Solicitudes 2014-2015',
                    'Admisiones 2015-2016',
                    'Solicitudes 2015-2016',
                    'Admisiones 2016-2017',
                    'Solicitudes 2016-2017',
                    'Población Primaria 2012-2013',
                    'Población Primaria 2013-2014',
                    'Población Primaria 2014-2015',
                    'Población Primaria 2015-2016',
                    'Población Primaria 2016-2017',
                    'Población Primaria 2017-2018']

In [22]:
df_total.to_csv("csv_files/DataSchools.csv", encoding = "utf-8")

In [ ]:
cols = ['Colegio',
        'Barrio',
        'Distrito',
        'Latitud',
        'Longitud',
        'Población Primaria 2012-2013',
        'Alumnos Primaria 2012-2013',
        'Solicitudes 2012-2013',
        'Admisiones 2012-2013',
        'Población Primaria 2013-2014',
        'Alumnos Primaria 2013-2014',
        'Solicitudes 2013-2014',
        'Admisiones 2013-2014',
        'Población Primaria 2014-2015',
        'Alumnos Primaria 2014-2015',
        'Solicitudes 2014-2015',
        'Admisiones 2014-2015',
        'Población Primaria 2015-2016',
        'Alumnos Primaria 2015-2016',
        'Solicitudes 2015-2016',
        'Admisiones 2015-2016',
        'Población Primaria 2016-2017',
        'Alumnos Primaria 2016-2017',
        'Solicitudes 2016-2017',
        'Admisiones 2016-2017',
        'Población Primaria 2017-2018']

In [ ]:
df_total = df_total[cols]